## 模拟退火算法

### 算法原理

模拟退火（simulated annealing, SA）的思想最早由Metropolis等提出，其出发点基于物理中固体物质的退火过程与一般的组合优化问题之间的相似性。

统计力学中，给物质加热并让其冷却再结晶的过程称为退火。从微观来看，退火过程实际上物质内部原子由高能量组态转换为低能量**组态**，表现在宏观上就是物质的状态不断发生变化，当冷却停止是变化也便停止。对于能量为 $E(s)$ 的组态 $s$ ，一个原子处于该组态的概率为

$$\Pr(s)=\frac{\displaystyle\exp\left(-\frac{E(s)}{kT}\right)}{\displaystyle\sum_w \exp \left(-\frac{E(w)}{kT}\right)}$$

其中 $k$ 为 $\textrm{Boltzmann}$ 常数， $T$ 为系统位于平衡状态时的温度。对于一个处于组态 $q$ 的原子，其下一刻转变为组态 $r$ 的概率为

$$\left\{\begin{aligned}
&\Pr(r|q)=1, & if\,\,E(r)<E(q)\\
&\Pr(r|q)=\exp\left(\frac{E(q)-E(r)}{kT}\right)& if\,\,E(r)\geq E(q)
\end{aligned}\tag{1}\right.$$

则当时间$t\rightarrow\infty$时，该原子处于某个组态$s$的概率收敛到Boltzmann分布。

模拟退火算法中假定初始状态即为高温，并以一定冷却方式（由冷却调度函数 $\alpha(T)$ 决定）进行冷却，冷却过程中每个原子组态 $q$ 对应一个候选解，所处组态的能量 $E(q)$ 可以类比为适应度或损失函数。在对组态进行更新时，对于随机选择的候选组态 $r$ ，当前组态依据式(1)进行替换。原子组态改变即寻找更好候选解的过程，当降到特定的温度或原子组态不再发生改变算法停止。

由(1)式可以得出，当温度越高时，越倾向于对可行域的搜索，当温度越低时，越倾向于对当前可行解的开发；自然退火与模拟退火算法之间类比如下：

自然退火|模拟退火
:--:|:--:
原子组态|候选解
组态能量|适应度
原子组态改变|重组
温度|搜索可行域的趋势
冷却|降低搜索力度的趋势

### 算法步骤

1. 定义目标函数；并初始化开始时温度 $T_0$ 、冷却调度函数 $\alpha(T)\in[0, T]$ ，以及最小化问题 $f(\boldsymbol{x})$ 的一个候选解 $\boldsymbol{x}_0$ ；

2. 随机生成一个候选解 $\boldsymbol{x}$ . 若$f(\boldsymbol{x_0})>f(\boldsymbol{x})$，则直接替换，否则以概率 $\exp\left[(f(\boldsymbol{x_0})-f(\boldsymbol{x}))/T\right]$ 替换原候选解；
或者以一种简化的概率替换，即以概率 $\exp\left[-c/T\right]$ 替换原候选解；

3. 更新温度： $T\leftarrow\alpha(T)$；

4. 判断是否满足终止条件，否则返回第2步；

### 算法优化

粗略分析不难得出，模拟退火算法中起始温度的选取、温度冷却的过程对整个算法在可行域的搜索能力有着很大的影响。若起始温度过高，则算法收敛速度变慢；若起始温度过低，则算法不能对可行域进行良好的探索。类似地，冷却调度 $\alpha(T)$ 降低过快，则来不急对可行域进行探索；而温度降低过慢依旧会导致收敛速度变慢的问题。冷却控制着从搜索解到开发解的过程。

#### 冷却调度

较为理想的冷却策略能够兼顾对解的搜索和开发，同时保证算法的收敛速度。常用的冷却策略包括线性冷却、指数冷却、逆冷却、对数冷却、逆线性冷却。

- **线性冷却**的冷却调度为
    $$\alpha(T^{(k)})=\max\{T_0-\eta k, T_{min}\}$$
    其中 $T_0$ 为初始温度，k为迭代次数，$\eta$ 为一取定常数，$T_{min}$ 为指定最低温度
- **逆冷却**的温度调度为
    $$\alpha(T^{(k)})=T^{(k)}/(1+\beta T^{(k)})=T_0/(1+k\beta T_0)$$
    其中$beta$是一个小的常数，通常取0.001的数量级。
- **指数冷却**下温度更新为
    $$\alpha(T^{(k)})=aT^{(k)}=a^{k+1}T_0$$
    通常取$a\in(0.8,1)$，显然a取值过小会导致冷却速度过快。
- **对数冷却**的冷却调度为
    $$\alpha(T^{(k)})=c/\ln(k+d)$$
    其中c, d均为取定常数，通常取d=1，k为迭代次数。对数冷却缺点在于在算法开始时快速下降，之后下降变得极为缓慢，故该冷却策略通常*收敛性很差*；然而可以从理论上证明（见《进化算法》P188），在某种条件下对数冷却调度能得到*全局最小*。
- **逆线性冷却**的温度调度为
    $$\alpha(T^{(k)})=T_0/k$$
    逆线性冷却会使得温度快速下降并迅速到达零度，即逆线性冷却更适合于*初值靠近已知最优解的问题*。同样地可以从理论上证明（见《进化算法》P190），在某种条件下逆线性冷却可以到达*全局最小*。
- **依赖维数的冷却**常用于不同维度需要不同的冷却速度的情况，例如函数
    $$f(\boldsymbol{x}) = 20 + e -20\exp\left(-0.2\sum_{i=1}^n y_i^2/n\right)-\exp\left(\sum_{i=1}^n (\cos2\pi y_i)/n\right)$$
    $$y_i=\left\{\begin{array}{ll}x_i\,,&i=2n-1\\x_i/4\,,&i=2n\\\end{array}\,,n=1, 2, \ldots\right.$$
    由于该函数在偶数维上更加平滑，从而应放缓温度降低的速率，以能够对可行域进行更好的探索，而对于变化更快的奇数维度，由于其对温度的变化较为敏感，温度下降减缓会导致收敛时间拉长，进而应采用较快降温的方式。这种方式的冷却主要通过算法实现：
    1. 初始化温度及冷却函数 $T_i\,,\,\alpha_i(T_i)\,,i=1, 2, \cdots, n$ ；初始化最小化问题 $f(\boldsymbol{x})$ 的一个候选解 $\boldsymbol{x}_0=[x_{01}, x_{02}, \cdots, x_{0n}]$；
    2. 生成候选解 $\boldsymbol{x}_1=[x_{11}, x_{12}, \cdots, x_{1n}]$. 若$f(\boldsymbol{x_0})>f(\boldsymbol{x}_1)$，则直接替换，否则对于每一维度i，以概率 $\exp\left[(f(\boldsymbol{x_0})-f(\boldsymbol{x}_1))/T_i\right]$ 将原候选解的第 i 维变量替换为新候选解第 i 维变量，即$x_{0i}\leftarrow x_{1i}$；
    3. 更新温度 $T_i = \alpha_i(T_i)\,,\,i=1, 2, \cdots, n$；相应冷却调度函数里的常数也应矢向量化为 n 维常向量；
    4. 判断是否满足终止条件，否则返回第 ii 步

#### 候选解的生成

在算法收敛到一个好的解之后，随机生成候选解的策略往往不足以使算法快速收敛，从而希望生成的候选解偏向当前的候选解 $\boldsymbol{x}_0$ ，故通常将随机数取为以 $\boldsymbol{x}_0$ 为中心的高斯随机数：

$$x_{1i}\leftarrow x_{0i}+N(0, T_{ki})\,,\,i =1, 2, \cdots,n$$

其中 $T_{ki}$ 指第 k 次迭代第 i 维的温度；对于各维度冷却规律一致的情况，上式退化为

$$\boldsymbol{x}_{1}\leftarrow\boldsymbol{x}_{0}+N(0, T_{k})$$

#### 重启

当温度下降过快时，算法容易陷入局部最优，故常采用的方法是在 L 次迭代后没有找到更好的候选解，若需要增加搜索力度，不妨重新初始化 $T_0$ 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class SA():
    def __init__(self, T0, search_space, dim:int = 2):
        self.T = [T0]
        self.dim = dim
        self.search_space = search_space  # 搜索空间半径
        self.atom = np.random.normal(loc=0., scale=search_space / 2, size=[1, self.dim])  # 初始化觅食蜂种群
        self.loss = []
        self.timer = 0

    # ---------------------目标函数Sphere函数-----------------------------
    def func(self, X):
        return np.e - 20 * np.exp(-0.2 * np.sqrt((X**2).sum() / len(X))) + np.exp(np.cos(2 * np.pi * X).sum() / len(X))

    # -------------------------迭代--------------------------------
    def iterate(self, max_iter, limit):
        for k in range(max_iter):
            atom1 = np.random.normal(loc=self.atom, scale=self.T[-1], size=[1, self.dim])
            if self.func(atom1) < self.func(self.atom):
                self.atom = atom1
                self.timer = 0
            else:
                r = np.random.rand()
                if r < np.exp(self.func(self.atom) - self.func(atom1) / self.T[-1]):
                    self.atom = atom1
                else:
                    self.timer += 1

            # # 线性冷却 T_k+1 = max(T_k - eta, T_min)；
            # # 不太稳定，eta >= 5有机会收敛；eta越大收敛机会也越大，但有时也会陷入局部极小值
            # self.T.append(np.max([self.T[-1] - 5, 4]))

            # # 指数冷却: T_k+1 = a T_k, 容易陷入局部极小值
            # self.T.append(self.T[-1] * 0.996)

            # # 逆冷却: T_k+1 = T_k / (1 + beta T_k), beta = 0.001效果好一些，但不稳定
            # self.T.append(self.T[-1] / (1 + 0.001 * self.T[-1]))

            # # 对数冷却 T_k = c / ln(k + d) ，也不稳定
            # self.T.append(self.T[0] * 0.8 / np.math.log(k + 2))

            # # T_k = T0 / k ，也不稳定
            self.T.append(self.T[0] / (k + 1))

            self.loss.append(self.func(self.atom))
            # 重启
            if self.timer > limit:
                self.T[-1] = self.T[0]
                self.timer = 0


sa = SA(100, 10, dim=2)
sa.iterate(max_iter=10000, limit=800)

loss = np.array(sa.loss) - np.min(sa.loss)
loss = loss / np.max(loss)

In [ ]:
plt.figure(1)
plt.title("Figure1")
plt.xlabel("iterators", size=14)
plt.ylabel("loss", size=14)
k = np.array(range(len(loss)))
plt.plot(k, loss, linewidth=3, color="b")
plt.ylim(0, np.max(loss)*17 / 16)
plt.figure(2)
plt.title("Figure2")
plt.xlabel("iterators", size=14)
plt.ylabel("temperature", size=14)
plt.plot(k, sa.T[1:], color="orange", linewidth=3)
plt.show()